In [1]:
import pandas as pd
import numpy as np

In [2]:
df = (
    pd.read_csv("../resources/application_train.csv", low_memory=False)
    # .sample(200_000, replace=True, random_state=0)
    .drop(columns=["SK_ID_CURR"])
)
df = df.loc[:, df.nunique() > 1]
# df = df.loc[:, df.nunique() < 250]


In [3]:
df.shape

(307511, 121)

In [4]:
df["TARGET"].value_counts(dropna=False)

0    282686
1     24825
Name: TARGET, dtype: int64

In [5]:
X = df.drop(columns=["TARGET"]).select_dtypes("number")
y = df["TARGET"]

In [6]:
X.nunique().value_counts().sort_index(ascending=False).head()

119831    1
114584    1
17460     1
15688     1
13672     1
dtype: int64

In [7]:
X.shape

(307511, 104)

In [8]:
# pd.Series(X.fillna(0).to_numpy().ravel(order="F")).to_csv(
#         "../../forust-profile/resources/contiguous_no_missing_no_sample.csv",
#         index=False,
#         header=False,
# )

# y.to_csv(
#     "../../forust-profile/resources/performance_no_sample.csv",
#     index=False,
#     header=False,
# )

In [8]:
from forust import GradientBooster
fmod = GradientBooster(
    iterations=500,
    learning_rate=0.3,
    max_depth=5,
    l2=1,
    min_leaf_weight=1.0,
    gamma=0.0,
    objective_type="LogLoss",
    # nbins=500,
    # parallel=False,
)
fmod.fit(X, y=y)

In [9]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y, fmod.predict(X)))
# print(roc_auc_score(y, xmod.predict(X, output_margin=True)))


0.8693096270370666


In [10]:
pd.Series(fmod.predict(X)).describe()

count    307511.000000
mean         -3.278663
std           3.882748
min        -110.479120
25%          -3.821643
50%          -3.013063
75%          -2.199183
max          45.951634
dtype: float64

In [15]:
print(fmod.text_dump()[0][0:1000])

0:leaf=-0.6932906,cover=7215.2573



In [11]:
# 0:[FLAG_DOCUMENT_13<1] yes=1,no=2,missing=2,gain=6.625,cover=72266.2266
# 	1:leaf=-0.691310763,cover=72011.4844
# 	2:leaf=-0.756317973,cover=254.744019


In [12]:
def grad(y: np.ndarray, y_hat: np.ndarray) -> np.ndarray:
    y_hat = 1 / (1 + np.exp(-y_hat))
    return y_hat - y

def hess(y: np.ndarray, y_hat: np.ndarray) -> np.ndarray:
    y_hat = 1 / (1 + np.exp(-y_hat))
    return y_hat * (1 - y_hat)

preds = np.ones(y.shape) * 0.5
h = hess(y, preds).astype(np.float32)
g = grad(y, preds)
print(h[0:10])
h.sum()


[0.23500371 0.23500371 0.23500371 0.23500371 0.23500371 0.23500371
 0.23500371 0.23500371 0.23500371 0.23500371]


72266.195

In [13]:
y.groupby(X.iloc[:,22] < 1).mean()

REG_REGION_NOT_WORK_REGION
False    0.088906
True     0.080291
Name: TARGET, dtype: float64

In [14]:
print(fmod.text_dump()[0][0:1000])

0:[20 < 21] yes=1,no=2,missing=2,gain=83469.72,cover=72127.15
      1:[88 < 1] yes=3,no=4,missing=4,gain=3939.9063,cover=72121.75
            3:[70 < 0.3804] yes=7,no=8,missing=7,gain=62295.938,cover=71986.62
                  7:[44 < 1] yes=11,no=12,missing=11,gain=24606.188,cover=71985.31
                        11:[86 < 1] yes=13,no=14,missing=14,gain=3845.5313,cover=71984.2
                              13:leaf=-0.69454795,cover=71887.68
                              14:leaf=1.191877,cover=96.52344
                        12:leaf=-33.067654,cover=1.1132813
                  8:leaf=-49.976654,cover=1.3085938
            4:leaf=0.9219857,cover=135.13281
      2:[28 < 0.1158262489348995] yes=5,no=6,missing=5,gain=165255.56,cover=5.3984375
            5:[2 < 405000] yes=9,no=10,missing=10,gain=4.205922,cover=4.230067
                  9:leaf=-0.55742013,cover=2.350037
                  10:leaf=0.002117185,cover=1.8800297
            6:leaf=-102.29872,cover=1.1683707



In [56]:
from forust.forust import percentiles

In [20]:
from xgboost import XGBClassifier
xmod = XGBClassifier(
    n_estimators=500,
    learning_rate=0.3,
    max_depth=5,
    reg_lambda=1,
    min_child_weight=1.0,
    gamma=0.0,
    objective="binary:logitraw",
    eval_metric="auc",
    tree_method="hist",
    # max_bin=1000,
)
xmod.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1.0,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, objective='binary:logitraw',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(max_iter=500)
hgb.fit(X, y)

HistGradientBoostingClassifier(max_iter=500)

In [22]:
from sklearn.metrics import roc_auc_score

In [23]:
roc_auc_score(y, fmod.predict(X))

0.8693096270370666

In [24]:
roc_auc_score(y, xmod.predict(X, output_margin=True))

0.9065143493042883

In [25]:
roc_auc_score(y, hgb.predict_proba(X)[:,1])

0.7809556871427288

In [26]:
print(xmod.get_booster().get_dump(with_stats=True)[0][0:1000])

0:[EXT_SOURCE_3<0.315472156] yes=1,no=2,missing=2,gain=1758.75,cover=72266.2266
	1:[EXT_SOURCE_2<0.380212724] yes=3,no=4,missing=4,gain=640.160156,cover=10567.1768
		3:[EXT_SOURCE_2<0.0765305534] yes=7,no=8,missing=8,gain=138.427246,cover=3018.85767
			7:[EXT_SOURCE_3<0.180887982] yes=15,no=16,missing=16,gain=36.8361816,cover=467.187378
				15:[AMT_CREDIT<269550] yes=31,no=32,missing=32,gain=15.6610718,cover=204.453232
					31:leaf=-0.359731078,cover=49.1157761
					32:leaf=-0.163963854,cover=155.337448
				16:[EXT_SOURCE_1<0.274144381] yes=33,no=34,missing=33,gain=20.3825989,cover=262.734161
					33:leaf=-0.330531746,cover=192.703049
					34:leaf=-0.525795937,cover=70.031105
			8:[EXT_SOURCE_3<0.149167463] yes=17,no=18,missing=18,gain=87.5302734,cover=2551.67017
				17:[EXT_SOURCE_1<0.405112624] yes=35,no=36,missing=35,gain=19.3948975,cover=694.905945
					35:leaf=-0.370736092,cover=564.948914
					36:leaf=-0.504486084,cover=129.957047
				18:[EXT_SOURCE_1<0.426402181] yes=37,no=38,mi

In [27]:
X.columns.to_series()[88]

'FLAG_DOCUMENT_12'

In [28]:
print(fmod.text_dump()[0][0:1000])

0:[20 < 21] yes=1,no=2,missing=2,gain=83469.72,cover=72127.15
      1:[88 < 1] yes=3,no=4,missing=4,gain=3939.9063,cover=72121.75
            3:[70 < 0.3804] yes=7,no=8,missing=7,gain=62295.938,cover=71986.62
                  7:[44 < 1] yes=11,no=12,missing=11,gain=24606.188,cover=71985.31
                        11:[86 < 1] yes=13,no=14,missing=14,gain=3845.5313,cover=71984.2
                              13:leaf=-0.69454795,cover=71887.68
                              14:leaf=1.191877,cover=96.52344
                        12:leaf=-33.067654,cover=1.1132813
                  8:leaf=-49.976654,cover=1.3085938
            4:leaf=0.9219857,cover=135.13281
      2:[28 < 0.1158262489348995] yes=5,no=6,missing=5,gain=165255.56,cover=5.3984375
            5:[2 < 405000] yes=9,no=10,missing=10,gain=4.205922,cover=4.230067
                  9:leaf=-0.55742013,cover=2.350037
                  10:leaf=0.002117185,cover=1.8800297
            6:leaf=-102.29872,cover=1.1683707



In [37]:
def gain(
        grad_sum: float,
        hess_sum: float,
        l2=1,
    ) -> float:
        return (grad_sum**2) / (hess_sum + l2)
root_gain = gain(g.sum(), h.sum())
m = (X.iloc[:, 20]<21)
gain_left = gain(g[m].sum(), h[m].sum())
gain_right = gain(g[~m].sum(), h[~m].sum())
root_gain, gain_left, gain_right
(gain_left + gain_right) - root_gain

-5.307436423958279

In [ ]:
pd.Series(xmod.predict(X, output_margin=True)).describe()

count    1000000.000000
mean          -3.237135
std            1.548699
min          -16.427555
25%           -4.132332
50%           -3.174613
75%           -2.263044
max            4.421914
dtype: float64